In [1]:
import numpy as np
import pandas as pd
from flask import Flask, request, jsonify, render_template
import pickle

In [2]:
app = Flask(__name__)
rf_model = pickle.load(open('rf_model.pkl', 'rb'))
recommender = pickle.load(open('recommender.pkl', 'rb'))
df = pickle.load(open('data.pkl', 'rb'))
vectorizer = pickle.load(open('tfidf_vect.pkl', 'rb'))

In [3]:
@app.route('/')
def home():
    return render_template('index.html')

In [4]:
@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    int_features = [x for x in request.form.values()]
    final_features = int_features[0]
    if final_features in recommender.index:
        d = recommender.loc[final_features].sort_values(ascending=False)[0:20] 
        out_df = d.to_frame()
        output = fineTune(out_df.index)
        output.index = np.arange(1, len(output) + 1)
        return render_template('index.html', prediction_text='Top 5 Recommended products are :',
        tables=[output.to_html(classes='data',header=False)], titles=output.columns.values)
    else:
        return render_template('index.html', prediction_text='Username not found ')

In [5]:
def fineTune(product_list,top=5):
    sample_data = df[df['name'].isin(product_list)][['review','name']].copy()
    #Using the vectorizer to transform the review data 
    sample_data_vect = vectorizer.transform(sample_data['review'])
    #Predicting sentiment using the final LogReg model
    sample_data['pred_sentiment'] = rf_model.predict(sample_data_vect)
    #Sorting based on the percentage positive sentiment
    final_out = sample_data.groupby(by='name').mean().sort_values('pred_sentiment',ascending=False)[:top]
    return pd.DataFrame(final_out.index).rename(columns={'name':'Product Name'})

In [6]:
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\MAhaDev\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
